In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import math
import random

MIN_LIST_LEN = 16
MAX_LIST_LEN = 16
MAX_STEPS = 640

SUCCESS_REWARD = 0.5
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.99
SHORTTERM_GAMMA = 0.7

EPS_START = 0.5
EPS_END = 0.1
EPS_DECAY = 1000
LR_SCHEDULER_GAMMA = 0.9
NUM_EPISODES = 3999
EPISODES_SAVE = 1000
OUTPUT_DIR = '/home/mcwave/code/autocode/datasets/rl_sort_transformer_easy/list16_transformer4_192_gamma07_step640_v3_data2'

# Define the vocabulary
# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    'less': 2,
    'equal': 3,
    'more': 4,
    '0': 5,
    '1': 6,
    '2': 7,
    '3': 8,
    '4': 9,
    '5': 10,
    '6': 11,
    '7': 12,
    '8': 13,
    '9': 14,
    '10': 15,
    '11': 16,
    '12': 17,
    '13': 18,
    '14': 19,
    '15': 20,
    'len1': 21,
    'len2': 22,
    'len3': 23,
    'len4': 24,
    'len5': 25,
    'len6': 26,
    'len7': 27,
    'len8': 28,
    'len9': 29,
    'len10': 30,
    'len11': 31,
    'len12': 32,
    'len13': 33,
    'len14': 34,
    'len15': 35,
    'len16': 36,
    'pos0': 37,
    'pos1': 38,
    'pos2': 39,
    'pos3': 40,
    'pos4': 41,
    'pos5': 42,
    'pos6': 43,
    'pos7': 44,
    'pos8': 45,
    'pos9': 46,
    'pos10': 47,
    'pos11': 48,
    'pos12': 49,
    'pos13': 50,
    'pos14': 51,
    'pos15': 52,
}
inv_vocab = {v: k for k, v in vocab.items()}

def compute_entropy(N, alpha=1):
    K = 2**N
    values = np.arange(K)
    unnormalized_probs = np.exp(-alpha * values)
    Z = unnormalized_probs.sum()
    probs = unnormalized_probs / Z
    return values, -np.log2(probs)

_, int_entropy = compute_entropy(4)

def get_entropy_of_integer(x):
    x = min(15, abs(x))
    return int_entropy[x]

def compute_min_delta_entropy(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = None

    # Iterate through each pair in the comparisons list
    i = len(comparisons) - 1
    xi, yi = comparisons[i]
    if i == 0:
        # For i = 0, use the first case directly
        min_delta = (xi, min(yi, yi - xi), 0)
    else:
        # For i > 0, compute all possible options and select the minimal one
        options = []

        # Simple Entropy
        simple_entropy = (xi, min(yi, yi - xi), 0)
        options.append(simple_entropy)

        # First Delta Entropy
        xi_prev, yi_prev = comparisons[i - 1]
        first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
        options.append(first_delta_entropy)

        # Second Delta Entropy (only valid for i > 1)
        if i > 1:
            xi_prev2, yi_prev2 = comparisons[i - 2]
            second_delta_entropy = (
                (xi - xi_prev) - (xi_prev - xi_prev2),
                (yi - yi_prev) - (yi_prev - yi_prev2),
                0,
            )
            options.append(second_delta_entropy)

        # Arbitrary Position Entropy (only valid for i > 1)
        for j in range(i):
            xj, yj = comparisons[j]
            arbitrary_position_entropy = (
                xi - xj,
                yi - yj,
                min(j, i - j),
            )
            options.append(arbitrary_position_entropy)

        # Find the option with the minimal sum
        min_delta = min(options, key=lambda t: sum([get_entropy_of_integer(x) for x in t]))

    entropy = sum([get_entropy_of_integer(x) for x in min_delta])
    if len(comparisons) == 1:
        return 3 * entropy
    else:
        return entropy

def generate_unique_random_list(length, min_value=1, max_value=100):
    """
    Generates a list of unique random integers within a specified range.
    If duplicates are detected, the list is regenerated.
    
    Args:
        length (int): The length of the list to generate.
        min_value (int): The minimum possible value (inclusive).
        max_value (int): The maximum possible value (inclusive).
        
    Returns:
        list: A list of unique random integers.
    """
    if length > (max_value - min_value + 1):
        raise ValueError("Length exceeds the range of unique values available.")
    
    while True:
        random_list = [random.randint(min_value, max_value) for _ in range(length)]
        if len(set(random_list)) == length:
            return random_list
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = generate_unique_random_list(self.length)
        self.sorted_list = sorted(self.list)
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None
        additional_response_tokens = []

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
            final_pos_1 = self.sorted_list.index(self.list[index1])
            final_pos_2 = self.sorted_list.index(self.list[index2])
            additional_response_tokens = [vocab[f"pos{final_pos_1}"], vocab[f"pos{final_pos_2}"]]
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy(), additional_response_tokens


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=640):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=192, nhead=8, num_layers=4):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
def compute_bellman_returns(raw_rewards, gamma):
    bellman_returns = []
    R = 0
    for r in raw_rewards[::-1]:
        R = r + gamma * R
        bellman_returns.insert(0, R)
    return bellman_returns

# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=2.3e-5)  # Reduced learning rate
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=LR_SCHEDULER_GAMMA)
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    f_training_data = open(os.path.join(OUTPUT_DIR, "action_sequence.txt"), "a")
    
    # Optionally, load a checkpoint
    #checkpoint_path = os.path.join('/home/mcwave/code/autocode/datasets/rl_sort_transformer_easy/list16_transformer4_192_gamma07_step640_v3', "ckpt_36000_0.6820_426.26.pth")
    #load_checkpoint(checkpoint_path, model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    success_step_cnts = []
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        output_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        comparisons = []
        steps = 0
        
        state = 'expect_action'
        done = False
        success = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
                print(comparisons)
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())
            output_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list, _ = env.step(action_tokens)
                    steps += 1
                    if done and reward == SUCCESS_REWARD:
                        success = True
                        if episode % 100 == 0:
                            print(action_sequence_str)
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                comparisons.append((int(inv_vocab[index1_token.item()]), 
                                    int(inv_vocab[index2_token.item()])))
                response_token, reward, done, current_list, additional_response_tokens = env.step(action_tokens)
                steps += 1
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                    output_tokens.append(response_token)
                    output_tokens.extend(additional_response_tokens)
                #
                if done and reward == SUCCESS_REWARD:
                    success = True
                    #if episode % 1 == 0:
                    #    print(1, decode(output_tokens, inv_vocab))
                else:
                    reward += COMPARISON_ENTROPY_MULTIPLIER * compute_min_delta_entropy(comparisons)
                #    
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        if success or len(output_tokens) >= MAX_STEPS // 3:
            action_sequence_str = decode(output_tokens, inv_vocab)
            f_training_data.write(action_sequence_str + (' success\n' if success else ' failure\n'))
            f_training_data.flush()
        # Post processing of each episode
        success_rewards = [0.0] * len(rewards)
        if success: 
            num_successes += 1
            success_rewards[-1] = SUCCESS_REWARD
            success_step_cnts.append(steps)
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns1 = compute_bellman_returns(rewards, SHORTTERM_GAMMA)
            returns2 = compute_bellman_returns(success_rewards, LONGTERM_GAMMA)
            returns = torch.tensor(np.array(returns1) + np.array(returns2)).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{steps}, total reward:{sum(rewards):.4f}, {t2-t1} sec")
            
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            #assert len(success_step_cnts) == num_successes
            success_rate = num_successes / episode_cnt
            avg_steps = sum(success_step_cnts) / len(success_step_cnts)
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}.pth")
            #
            # Reduce the lr
            scheduler.step()
            # Optionally, log the learning rate
            current_lr = scheduler.get_last_lr()[0]
            print(f"Learning rate = {current_lr:.6f}")

if __name__ == "__main__":
    for i in range(100):
        train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss:-38.2653, fail, steps:5, total reward:-10.9000, 0.3607790470123291 sec
Episode 1, loss:-8.3475, fail, steps:0, total reward:-10.0000, 0.005465507507324219 sec
Episode 2, loss:-10.3944, fail, steps:0, total reward:-10.0000, 0.0035965442657470703 sec
Episode 3, loss:-10.5880, fail, steps:0, total reward:-10.0000, 0.003523111343383789 sec
Episode 4, loss:-52.9078, fail, steps:6, total reward:-13.2000, 0.03955698013305664 sec
Episode 5, loss:-40.8068, fail, steps:6, total reward:-9.2000, 0.037806034088134766 sec
Episode 6, loss:-49.0015, fail, steps:6, total reward:-11.9000, 0.032317399978637695 sec
Episode 7, loss:-12.3311, fail, steps:0, total reward:-10.0000, 0.0038008689880371094 sec
Episode 8, loss:-85.2387, fail, steps:32, total reward:-15.9000, 0.23056602478027344 sec
Episode 9, loss:-68.7264, fail, steps:21, total reward:-15.8000, 0.14406466484069824 sec
Episode 10, loss:-49.2453, fail, steps:5, total reward:-10.2000, 0.03914690017700195 sec
Episode 11, loss:-60.467

Episode 92, loss:-225.3179, fail, steps:167, total reward:-43.1000, 1.5502398014068604 sec
Episode 93, loss:-238.0742, fail, steps:166, total reward:-45.4000, 1.6021130084991455 sec
Episode 94, loss:-235.8932, fail, steps:165, total reward:-44.4000, 1.6619157791137695 sec
Episode 95, loss:-224.5884, fail, steps:163, total reward:-42.4000, 1.7528071403503418 sec
Episode 96, loss:-224.0664, fail, steps:163, total reward:-43.7000, 1.6472992897033691 sec
Episode 97, loss:-231.1882, fail, steps:163, total reward:-44.4000, 1.7468502521514893 sec
Episode 98, loss:-105.2179, fail, steps:16, total reward:-15.5000, 0.21950030326843262 sec
Episode 99, loss:-261.6533, fail, steps:163, total reward:-49.7000, 1.6017308235168457 sec
Episode 100, loss:-192.7031, fail, steps:103, total reward:-35.8000, 1.1120624542236328 sec
Episode 101, loss:-226.9781, fail, steps:165, total reward:-44.4000, 1.8763034343719482 sec
Episode 102, loss:-222.0359, fail, steps:163, total reward:-43.7000, 1.6284630298614502 

Episode 182, loss:-159.4192, fail, steps:190, total reward:-17.0000, 1.6180195808410645 sec
Episode 183, loss:-150.5426, fail, steps:185, total reward:-17.3000, 1.6746015548706055 sec
Episode 184, loss:-178.3995, fail, steps:161, total reward:-16.9000, 1.5215263366699219 sec
Episode 185, loss:-149.5165, fail, steps:179, total reward:-19.9000, 1.7660398483276367 sec
Episode 186, loss:-156.5082, fail, steps:184, total reward:-18.3000, 1.7295520305633545 sec
Episode 187, loss:-159.6913, fail, steps:186, total reward:-18.3000, 1.736706256866455 sec
Episode 188, loss:-147.0306, fail, steps:134, total reward:-17.1000, 1.2752232551574707 sec
Episode 189, loss:-159.4322, fail, steps:181, total reward:-20.6000, 1.915752649307251 sec
Episode 190, loss:-192.3842, fail, steps:130, total reward:-29.6000, 1.2943730354309082 sec
Episode 191, loss:-148.1719, fail, steps:152, total reward:-16.0000, 1.289625644683838 sec
Episode 192, loss:-164.4045, fail, steps:182, total reward:-19.6000, 1.782850503921

Episode 272, loss:-40.6702, fail, steps:31, total reward:-3.7000, 0.30789661407470703 sec
Episode 273, loss:-73.5686, fail, steps:52, total reward:-10.0000, 0.394561767578125 sec
Episode 274, loss:-121.2358, fail, steps:164, total reward:-11.2000, 1.2836782932281494 sec
Episode 275, loss:-123.1524, fail, steps:190, total reward:-11.0000, 1.8239924907684326 sec
Episode 276, loss:-80.8452, fail, steps:95, total reward:-6.0000, 0.789050817489624 sec
Episode 277, loss:-31.0075, fail, steps:0, total reward:-10.0000, 0.03280448913574219 sec
Episode 278, loss:-164.2621, fail, steps:180, total reward:-22.9000, 1.460789442062378 sec
Episode 279, loss:-121.8169, fail, steps:187, total reward:-13.3000, 1.6609852313995361 sec
Episode 280, loss:-146.9718, fail, steps:182, total reward:-17.6000, 1.6702094078063965 sec
Episode 281, loss:-117.2205, fail, steps:187, total reward:-11.3000, 1.7165963649749756 sec
Episode 282, loss:-96.5409, fail, steps:190, total reward:-5.7000, 1.7461426258087158 sec
Ep

...

Episode 526, loss:-81.5535, fail, steps:193, total reward:-2.7000, 1.7131762504577637 sec
Episode 527, loss:-90.4073, fail, steps:187, total reward:-8.0000, 1.832181692123413 sec
Episode 528, loss:-53.4347, fail, steps:196, total reward:3.6000, 1.9628643989562988 sec
Episode 529, loss:-35.8987, fail, steps:202, total reward:10.2000, 1.8990986347198486 sec
Episode 530, loss:-123.7586, fail, steps:181, total reward:-16.6000, 1.8802731037139893 sec
Episode 531, loss:-123.8752, fail, steps:185, total reward:-13.3000, 1.8734490871429443 sec
Episode 532, loss:-85.7976, fail, steps:192, total reward:-3.7000, 1.998791217803955 sec
Episode 533, loss:-63.1202, fail, steps:32, total reward:-6.6000, 0.33876538276672363 sec
Episode 534, loss:-60.1469, fail, steps:196, total reward:3.6000, 1.6993889808654785 sec
Episode 535, loss:-93.6755, fail, steps:188, total reward:-7.0000, 1.7820379734039307 sec
Episode 536, loss:-88.4905, fail, steps:188, total reward:-7.0000, 1.7961952686309814 sec
Episode 53

Episode 618, loss:-93.1558, fail, steps:186, total reward:-10.3000, 1.910548210144043 sec
Episode 619, loss:-95.0680, fail, steps:188, total reward:-7.0000, 1.9244005680084229 sec
Episode 620, loss:-46.6317, fail, steps:75, total reward:1.3000, 0.6503927707672119 sec
Episode 621, loss:-124.0742, fail, steps:190, total reward:-11.0000, 1.7218871116638184 sec
Episode 622, loss:-104.8855, fail, steps:131, total reward:-9.0000, 1.258127212524414 sec
Episode 623, loss:-84.6433, fail, steps:190, total reward:-5.7000, 2.1843137741088867 sec
Episode 624, loss:-81.5464, fail, steps:191, total reward:-6.7000, 1.9445524215698242 sec
Episode 625, loss:-27.8061, succeed, steps:152, total reward:-9.1000, 1.5881271362304688 sec
Episode 626, loss:-77.0434, fail, steps:190, total reward:-3.7000, 2.0332584381103516 sec
Episode 627, loss:-50.3730, fail, steps:196, total reward:3.6000, 1.8740143775939941 sec
Episode 628, loss:-96.2973, fail, steps:189, total reward:-8.0000, 2.020362138748169 sec
Episode 6

Episode 710, loss:-66.9263, fail, steps:192, total reward:-1.7000, 1.8180983066558838 sec
Episode 711, loss:-58.3141, fail, steps:195, total reward:2.6000, 1.8913447856903076 sec
Episode 712, loss:-71.9323, fail, steps:189, total reward:-6.0000, 1.9292116165161133 sec
Episode 713, loss:-55.8843, fail, steps:35, total reward:-4.9000, 0.3465259075164795 sec
Episode 714, loss:-75.8927, fail, steps:188, total reward:-7.0000, 1.6365861892700195 sec
Episode 715, loss:-66.0403, fail, steps:12, total reward:-7.1000, 0.1400902271270752 sec
Episode 716, loss:-50.6055, fail, steps:195, total reward:2.6000, 1.7335643768310547 sec
Episode 717, loss:-78.5280, fail, steps:192, total reward:-3.7000, 1.794978380203247 sec
Episode 718, loss:-85.6282, fail, steps:188, total reward:-7.0000, 1.8516161441802979 sec
Episode 719, loss:-102.5013, fail, steps:188, total reward:-9.0000, 1.847726821899414 sec
Episode 720, loss:-71.3721, fail, steps:195, total reward:0.6000, 1.8644397258758545 sec
Episode 721, los

Episode 802, loss:-78.8059, fail, steps:187, total reward:-9.3000, 1.934786319732666 sec
Episode 803, loss:-58.8424, fail, steps:192, total reward:-1.7000, 1.9367856979370117 sec
Episode 804, loss:-66.1171, fail, steps:190, total reward:-5.0000, 1.8941750526428223 sec
Episode 805, loss:-69.7092, fail, steps:64, total reward:-7.1000, 0.6151382923126221 sec
Episode 806, loss:-94.8892, fail, steps:188, total reward:-9.0000, 1.7160065174102783 sec
Episode 807, loss:-71.0240, fail, steps:190, total reward:-3.7000, 1.8775115013122559 sec
Episode 808, loss:-107.7008, fail, steps:184, total reward:-15.6000, 1.874577283859253 sec
Episode 809, loss:-74.2032, fail, steps:187, total reward:-9.3000, 1.90771484375 sec
Episode 810, loss:-97.4951, fail, steps:186, total reward:-12.3000, 1.9608449935913086 sec
Episode 811, loss:-99.3712, fail, steps:94, total reward:-5.7000, 0.9692862033843994 sec
Episode 812, loss:-63.4836, fail, steps:191, total reward:-2.7000, 2.360426425933838 sec
Episode 813, loss

Episode 894, loss:-7.6261, fail, steps:205, total reward:16.5000, 1.9118473529815674 sec
Episode 895, loss:-72.9960, fail, steps:188, total reward:-7.0000, 1.9717926979064941 sec
Episode 896, loss:-18.7557, fail, steps:202, total reward:13.5000, 1.942922830581665 sec
Episode 897, loss:-65.5595, fail, steps:189, total reward:-6.0000, 1.925945520401001 sec
